In [ ]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors

In [ ]:
# FCC dataset is already loaded in Colab, else load manually:
# books = pd.read_csv('books.csv')
# ratings = pd.read_csv('ratings.csv')

In [ ]:
# Remove users with <200 ratings
user_counts = ratings['User-ID'].value_counts()
ratings = ratings[ratings['User-ID'].isin(user_counts[user_counts >= 200].index)]

# Remove books with <100 ratings
book_counts = ratings['ISBN'].value_counts()
ratings = ratings[ratings['ISBN'].isin(book_counts[book_counts >= 100].index)]

# Merge datasets
df = ratings.merge(books, on='ISBN')

# Create pivot matrix: Book-Title x User-ID
pivot = df.pivot_table(index='Book-Title', columns='User-ID', values='Book-Rating').fillna(0)
book_matrix = pivot.values
book_list = pivot.index.tolist()

In [ ]:
model = NearestNeighbors(metric='cosine', algorithm='brute')
model.fit(book_matrix)

In [ ]:
def get_recommends(book_title):
    if book_title not in book_list:
        raise ValueError('Book title not found in dataset')
    idx = book_list.index(book_title)
    distances, indices = model.kneighbors([book_matrix[idx]], n_neighbors=6)
    recs = []
    for i in range(1, 6):
        recs.append([book_list[indices[0][i]], float(distances[0][i])])
    return [book_title, recs]

In [ ]:
get_recommends('The Queen of the Damned (Vampire Chronicles (Paperback))')